In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    ################################################ s', r = env(s, a)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    next_states_logits, Qlogits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                      action_size=action_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss_ = tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, labels=next_states_labels)
    eloss = tf.reduce_mean(tf.reduce_sum(eloss_, axis=1))
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, nextQlogits = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                          action_size=action_size, state_size=state_size, reuse=True)
    #################################################### GAN
    Qlogits = tf.reshape(Qlogits, shape=[-1])
    nextQlogits = tf.reshape(nextQlogits, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(nextQlogits) # minimize nextQ
    aloss = -tf.reduce_mean(nextQlogits) # maximize nextQ
    ###################################################### Qlearning
    ###################################################### Q(s,a)= r + Q'(s',a')
    rewards = tf.reduce_mean(tf.square(Qlogits - (gamma * nextQlogits)))
    ##################################################### GAN + DDPG
    eloss += tf.reduce_mean(rewards) # minimize entropy=error
    aloss += -tf.reduce_mean(rewards) # maximize entropy=rewards
    return actions_logits, aloss, eloss, rewards

In [7]:
def model_opt(a_loss, e_loss, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
    return a_opt, e_opt

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.rewards_ = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt = model_opt(a_loss=self.a_loss, e_loss=self.e_loss,
                                           a_learning_rate=a_learning_rate,
                                           e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        #print(done)
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [14]:
def minibacth(memory):
    # Training with the maxrated minibatch
    batch = memory.buffer
    #for idx in range(memory_size// batch_size):
    while True:
        idx = np.random.choice(np.arange(memory_size// batch_size))
        states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        states = states[rates >= np.max(rates)]
        actions = actions[rates >= np.max(rates)]
        next_states = next_states[rates >= np.max(rates)]
        rewards = rewards[rates >= np.max(rates)]
        dones = dones[rates >= np.max(rates)]
        rates = rates[rates >= np.max(rates)]
        if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
            break
    return states, actions, next_states, rewards, dones, rates

In [15]:
def minibacth_random(memory, batch_size):
    # Training with the maxrated minibatch
    while True:
        batch = memory.sample(batch_size=batch_size)
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        next_states = np.array([each[2] for each in batch])
        rewards = np.array([each[3] for each in batch])
        dones = np.array([each[4] for each in batch])
        rates = np.array([each[5] for each in batch])
        states = states[rates > 0]
        actions = actions[rates > 0]
        next_states = next_states[rates > 0]
        rewards = rewards[rates > 0]
        dones = dones[rates > 0]
        rates = rates[rates > 0]
        if len(dones) > 0:
            break
    return states, actions, next_states, rewards, dones, rates

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        rate = -1

        # Training steps/batches
        for num_step in range(11111):
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            total_reward += reward
            state = next_state
            
            # Online policy/on-policy/online training
            states, actions, next_states, rewards, dones, rates = minibacth_random(memory=memory, 
                                                                                   batch_size=batch_size)
            feed_dict = {model.states: states, model.actions: actions, model.next_states: next_states,
                         model.rewards: rewards, model.dones: dones, model.rates: rates}
            eloss, _ = sess.run([model.e_loss, model.e_opt], feed_dict)
            aloss, _ = sess.run([model.a_loss, model.a_opt], feed_dict)
            eloss_batch.append(eloss)
            aloss_batch.append(aloss)
            # End of episode/result/success rate available
            if done is True:
                # Rating the latest played episode
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                break

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        
        # Break episode/epoch loop
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of 500 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:14.0000 R:14.0000 rate:0.0280 aloss:-0.2414 eloss:4.0268 exploreP:0.9986
Episode:1 meanR:21.5000 R:29.0000 rate:0.0580 aloss:-0.2199 eloss:3.9907 exploreP:0.9958
Episode:2 meanR:26.6667 R:37.0000 rate:0.0740 aloss:-0.1883 eloss:3.9373 exploreP:0.9921
Episode:3 meanR:23.2500 R:13.0000 rate:0.0260 aloss:-0.1646 eloss:3.8975 exploreP:0.9908
Episode:4 meanR:24.0000 R:27.0000 rate:0.0540 aloss:-0.1465 eloss:3.8688 exploreP:0.9882
Episode:5 meanR:23.8333 R:23.0000 rate:0.0460 aloss:-0.1247 eloss:3.8320 exploreP:0.9859
Episode:6 meanR:22.2857 R:13.0000 rate:0.0260 aloss:-0.1085 eloss:3.8042 exploreP:0.9847
Episode:7 meanR:23.5000 R:32.0000 rate:0.0640 aloss:-0.0910 eloss:3.7756 exploreP:0.9816
Episode:8 meanR:22.2222 R:12.0000 rate:0.0240 aloss:-0.0728 eloss:3.7455 exploreP:0.9804
Episode:9 meanR:24.3000 R:43.0000 rate:0.0860 aloss:-0.0512 eloss:3.7100 exploreP:0.9762
Episode:10 meanR:23.2727 R:13.0000 rate:0.0260 aloss:-0.0301 eloss:3.6745 exploreP:0.9750
Episode:11 meanR:25.

Episode:93 meanR:20.3936 R:26.0000 rate:0.0520 aloss:0.3524 eloss:3.0556 exploreP:0.8273
Episode:94 meanR:20.3895 R:20.0000 rate:0.0400 aloss:0.3648 eloss:3.0394 exploreP:0.8257
Episode:95 meanR:20.2604 R:8.0000 rate:0.0160 aloss:0.3674 eloss:3.0346 exploreP:0.8250
Episode:96 meanR:20.1959 R:14.0000 rate:0.0280 aloss:0.3818 eloss:3.0179 exploreP:0.8239
Episode:97 meanR:20.1735 R:18.0000 rate:0.0360 aloss:0.3848 eloss:3.0123 exploreP:0.8224
Episode:98 meanR:20.2020 R:23.0000 rate:0.0460 aloss:0.4046 eloss:2.9895 exploreP:0.8205
Episode:99 meanR:20.0900 R:9.0000 rate:0.0180 aloss:0.4077 eloss:2.9835 exploreP:0.8198
Episode:100 meanR:20.2900 R:34.0000 rate:0.0680 aloss:0.4214 eloss:2.9658 exploreP:0.8171
Episode:101 meanR:20.1700 R:17.0000 rate:0.0340 aloss:0.4373 eloss:2.9451 exploreP:0.8157
Episode:102 meanR:19.9000 R:10.0000 rate:0.0200 aloss:0.4386 eloss:2.9422 exploreP:0.8149
Episode:103 meanR:19.9500 R:18.0000 rate:0.0360 aloss:0.4540 eloss:2.9237 exploreP:0.8134
Episode:104 meanR:1

Episode:185 meanR:17.5300 R:23.0000 rate:0.0460 aloss:1.3986 eloss:1.8276 exploreP:0.7067
Episode:186 meanR:17.8400 R:46.0000 rate:0.0920 aloss:1.4108 eloss:1.8166 exploreP:0.7035
Episode:187 meanR:17.8100 R:14.0000 rate:0.0280 aloss:1.4405 eloss:1.7883 exploreP:0.7026
Episode:188 meanR:17.9600 R:38.0000 rate:0.0760 aloss:1.4507 eloss:1.7791 exploreP:0.6999
Episode:189 meanR:17.9600 R:12.0000 rate:0.0240 aloss:1.4933 eloss:1.7363 exploreP:0.6991
Episode:190 meanR:18.0200 R:16.0000 rate:0.0320 aloss:1.5083 eloss:1.7227 exploreP:0.6980
Episode:191 meanR:17.7400 R:16.0000 rate:0.0320 aloss:1.4935 eloss:1.7405 exploreP:0.6969
Episode:192 meanR:17.7800 R:14.0000 rate:0.0280 aloss:1.5145 eloss:1.7194 exploreP:0.6960
Episode:193 meanR:17.6500 R:13.0000 rate:0.0260 aloss:1.5045 eloss:1.7290 exploreP:0.6951
Episode:194 meanR:17.5500 R:10.0000 rate:0.0200 aloss:1.5280 eloss:1.7075 exploreP:0.6944
Episode:195 meanR:17.6000 R:13.0000 rate:0.0260 aloss:1.5564 eloss:1.6776 exploreP:0.6935
Episode:19

Episode:278 meanR:17.9700 R:10.0000 rate:0.0200 aloss:2.6527 eloss:0.8095 exploreP:0.5988
Episode:279 meanR:17.9700 R:14.0000 rate:0.0280 aloss:2.6799 eloss:0.7855 exploreP:0.5979
Episode:280 meanR:17.9400 R:9.0000 rate:0.0180 aloss:2.7247 eloss:0.7480 exploreP:0.5974
Episode:281 meanR:17.8700 R:13.0000 rate:0.0260 aloss:2.6839 eloss:0.7828 exploreP:0.5966
Episode:282 meanR:17.9800 R:25.0000 rate:0.0500 aloss:2.6621 eloss:0.8126 exploreP:0.5952
Episode:283 meanR:17.9600 R:12.0000 rate:0.0240 aloss:2.6880 eloss:0.7895 exploreP:0.5945
Episode:284 meanR:18.0200 R:22.0000 rate:0.0440 aloss:2.6755 eloss:0.8016 exploreP:0.5932
Episode:285 meanR:18.0400 R:25.0000 rate:0.0500 aloss:2.7068 eloss:0.7722 exploreP:0.5917
Episode:286 meanR:17.6800 R:10.0000 rate:0.0200 aloss:2.7675 eloss:0.7114 exploreP:0.5911
Episode:287 meanR:17.6500 R:11.0000 rate:0.0220 aloss:2.7642 eloss:0.7232 exploreP:0.5905
Episode:288 meanR:17.3700 R:10.0000 rate:0.0200 aloss:2.7092 eloss:0.7816 exploreP:0.5899
Episode:289

Episode:371 meanR:14.3800 R:18.0000 rate:0.0360 aloss:3.3503 eloss:0.2057 exploreP:0.5266
Episode:372 meanR:14.3900 R:13.0000 rate:0.0260 aloss:3.3011 eloss:0.2593 exploreP:0.5259
Episode:373 meanR:14.3200 R:10.0000 rate:0.0200 aloss:3.3827 eloss:0.1800 exploreP:0.5254
Episode:374 meanR:14.0900 R:9.0000 rate:0.0180 aloss:3.4629 eloss:0.1039 exploreP:0.5249
Episode:375 meanR:14.0700 R:11.0000 rate:0.0220 aloss:3.3901 eloss:0.1762 exploreP:0.5244
Episode:376 meanR:13.8000 R:10.0000 rate:0.0200 aloss:3.3425 eloss:0.2179 exploreP:0.5238
Episode:377 meanR:13.8300 R:12.0000 rate:0.0240 aloss:3.3827 eloss:0.1799 exploreP:0.5232
Episode:378 meanR:13.8400 R:11.0000 rate:0.0220 aloss:3.4075 eloss:0.1627 exploreP:0.5227
Episode:379 meanR:13.8400 R:14.0000 rate:0.0280 aloss:3.3560 eloss:0.2092 exploreP:0.5219
Episode:380 meanR:13.8800 R:13.0000 rate:0.0260 aloss:3.4614 eloss:0.1112 exploreP:0.5213
Episode:381 meanR:13.8400 R:9.0000 rate:0.0180 aloss:3.4063 eloss:0.1658 exploreP:0.5208
Episode:382 

...

Episode:736 meanR:16.6500 R:21.0000 rate:0.0420 aloss:2.4189 eloss:0.7928 exploreP:0.3238
Episode:737 meanR:17.3100 R:77.0000 rate:0.1540 aloss:2.6644 eloss:0.5365 exploreP:0.3213
Episode:738 meanR:17.6300 R:42.0000 rate:0.0840 aloss:2.9716 eloss:0.2197 exploreP:0.3200
Episode:739 meanR:17.9000 R:43.0000 rate:0.0860 aloss:3.1220 eloss:0.0640 exploreP:0.3187
Episode:740 meanR:18.0800 R:29.0000 rate:0.0580 aloss:3.2248 eloss:-0.0416 exploreP:0.3178
Episode:741 meanR:18.3400 R:35.0000 rate:0.0700 aloss:3.3332 eloss:-0.1526 exploreP:0.3167
Episode:742 meanR:18.5200 R:27.0000 rate:0.0540 aloss:3.4740 eloss:-0.2926 exploreP:0.3159
Episode:743 meanR:18.6300 R:21.0000 rate:0.0420 aloss:3.5387 eloss:-0.3605 exploreP:0.3153
Episode:744 meanR:18.7600 R:25.0000 rate:0.0500 aloss:3.6719 eloss:-0.4888 exploreP:0.3145
Episode:745 meanR:18.8700 R:26.0000 rate:0.0520 aloss:3.6864 eloss:-0.5015 exploreP:0.3137
Episode:746 meanR:19.0800 R:32.0000 rate:0.0640 aloss:3.7556 eloss:-0.5672 exploreP:0.3127
Epi

Episode:827 meanR:32.2100 R:47.0000 rate:0.0940 aloss:4.7874 eloss:-1.5602 exploreP:0.2436
Episode:828 meanR:32.1300 R:20.0000 rate:0.0400 aloss:4.7463 eloss:-1.5164 exploreP:0.2431
Episode:829 meanR:32.4700 R:70.0000 rate:0.1400 aloss:4.6817 eloss:-1.4540 exploreP:0.2415
Episode:830 meanR:32.5100 R:29.0000 rate:0.0580 aloss:4.7752 eloss:-1.5525 exploreP:0.2408
Episode:831 meanR:32.5100 R:32.0000 rate:0.0640 aloss:4.7216 eloss:-1.5012 exploreP:0.2401
Episode:832 meanR:32.4000 R:30.0000 rate:0.0600 aloss:4.6447 eloss:-1.4249 exploreP:0.2394
Episode:833 meanR:32.3600 R:30.0000 rate:0.0600 aloss:4.5881 eloss:-1.3720 exploreP:0.2387
Episode:834 meanR:32.5900 R:38.0000 rate:0.0760 aloss:4.7095 eloss:-1.5002 exploreP:0.2379
Episode:835 meanR:32.8300 R:63.0000 rate:0.1260 aloss:4.6924 eloss:-1.4804 exploreP:0.2364
Episode:836 meanR:33.2100 R:59.0000 rate:0.1180 aloss:4.6476 eloss:-1.4401 exploreP:0.2351
Episode:837 meanR:32.8300 R:39.0000 rate:0.0780 aloss:4.7197 eloss:-1.5129 exploreP:0.2342

Episode:918 meanR:39.1900 R:22.0000 rate:0.0440 aloss:5.1536 eloss:-1.9189 exploreP:0.1727
Episode:919 meanR:39.1200 R:22.0000 rate:0.0440 aloss:5.0194 eloss:-1.7811 exploreP:0.1724
Episode:920 meanR:38.9700 R:28.0000 rate:0.0560 aloss:5.1057 eloss:-1.8676 exploreP:0.1719
Episode:921 meanR:38.8500 R:26.0000 rate:0.0520 aloss:4.9898 eloss:-1.7544 exploreP:0.1715
Episode:922 meanR:38.7300 R:28.0000 rate:0.0560 aloss:4.9468 eloss:-1.7121 exploreP:0.1710
Episode:923 meanR:38.6800 R:16.0000 rate:0.0320 aloss:5.0326 eloss:-1.7958 exploreP:0.1708
Episode:924 meanR:38.6000 R:24.0000 rate:0.0480 aloss:4.7472 eloss:-1.5168 exploreP:0.1704
Episode:925 meanR:38.5200 R:17.0000 rate:0.0340 aloss:5.0525 eloss:-1.8221 exploreP:0.1701
Episode:926 meanR:38.4800 R:24.0000 rate:0.0480 aloss:4.9862 eloss:-1.7533 exploreP:0.1697
Episode:927 meanR:38.2900 R:28.0000 rate:0.0560 aloss:5.2382 eloss:-2.0028 exploreP:0.1693
Episode:928 meanR:38.3600 R:27.0000 rate:0.0540 aloss:4.9443 eloss:-1.7088 exploreP:0.1689

Episode:1008 meanR:22.1700 R:22.0000 rate:0.0440 aloss:4.8502 eloss:-1.6277 exploreP:0.1447
Episode:1009 meanR:22.0800 R:20.0000 rate:0.0400 aloss:4.6534 eloss:-1.4317 exploreP:0.1445
Episode:1010 meanR:21.9500 R:14.0000 rate:0.0280 aloss:4.5794 eloss:-1.3595 exploreP:0.1443
Episode:1011 meanR:21.7000 R:15.0000 rate:0.0300 aloss:4.5810 eloss:-1.3605 exploreP:0.1441
Episode:1012 meanR:21.6100 R:17.0000 rate:0.0340 aloss:4.7503 eloss:-1.5341 exploreP:0.1439
Episode:1013 meanR:21.3500 R:15.0000 rate:0.0300 aloss:5.1011 eloss:-1.8799 exploreP:0.1437
Episode:1014 meanR:21.1000 R:17.0000 rate:0.0340 aloss:4.8075 eloss:-1.5893 exploreP:0.1434
Episode:1015 meanR:20.9300 R:20.0000 rate:0.0400 aloss:4.8134 eloss:-1.5936 exploreP:0.1432
Episode:1016 meanR:20.7800 R:20.0000 rate:0.0400 aloss:4.8053 eloss:-1.5836 exploreP:0.1429
Episode:1017 meanR:20.6600 R:20.0000 rate:0.0400 aloss:4.9492 eloss:-1.7277 exploreP:0.1426
Episode:1018 meanR:20.6500 R:21.0000 rate:0.0420 aloss:5.0051 eloss:-1.7835 expl

Episode:1098 meanR:24.0100 R:18.0000 rate:0.0360 aloss:5.0582 eloss:-1.8335 exploreP:0.1187
Episode:1099 meanR:23.8800 R:12.0000 rate:0.0240 aloss:5.2581 eloss:-2.0286 exploreP:0.1186
Episode:1100 meanR:23.8600 R:16.0000 rate:0.0320 aloss:4.8759 eloss:-1.6475 exploreP:0.1184
Episode:1101 meanR:23.7900 R:17.0000 rate:0.0340 aloss:4.8899 eloss:-1.6692 exploreP:0.1182
Episode:1102 meanR:23.6200 R:14.0000 rate:0.0280 aloss:4.6591 eloss:-1.4323 exploreP:0.1181
Episode:1103 meanR:23.6000 R:17.0000 rate:0.0340 aloss:4.5613 eloss:-1.3400 exploreP:0.1179
Episode:1104 meanR:23.4700 R:20.0000 rate:0.0400 aloss:4.8527 eloss:-1.6259 exploreP:0.1177
Episode:1105 meanR:23.2200 R:19.0000 rate:0.0380 aloss:4.8450 eloss:-1.6214 exploreP:0.1175
Episode:1106 meanR:23.2600 R:23.0000 rate:0.0460 aloss:4.8252 eloss:-1.6022 exploreP:0.1172
Episode:1107 meanR:23.3900 R:32.0000 rate:0.0640 aloss:4.7198 eloss:-1.4956 exploreP:0.1169
Episode:1108 meanR:23.3600 R:19.0000 rate:0.0380 aloss:4.6561 eloss:-1.4398 expl

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
